In [1]:
import re

entry_string = '''@book{chomsky1965,
author={Chomsky, Noam},
year={1965},
title={Aspects of the {Theory} of {Syntax}},
address={Cambridge, {MA}},
publisher={MIT {Press}}}
'''

entry_dict = dict()
type_key_rgx = r'@(?P<type>.+)\{(?P<key>.+),'
properties_rgx = r'\s*(?P<prp>.+)\s?=\s?\{?(?P<val>.+?)\}[,\}$]'
re.search(type_key_rgx, entry_string).group('key')

'chomsky1965'

In [2]:
?re.search

In [3]:
# entry_dict['type'] = re.search(type_key_rgx, entry_string).group('type')
# entry_dict['key'] = re.search(type_key_rgx, entry_string).group('key')

entry_string = '''@article{yip1988,
  title={Template morphology and the direction of association},
  author={Yip, Moira},
  journal={Natural {Language} \& {Linguistic} {Theory}},
  volume={6},
  number={4},
  pages={551--577},
  year={1988},
  publisher={Springer},
  location={New York}
}'''

properties_rgx = r'\s*(?P<prp>.+)\s?=\s?\{?(?P<val>.+?)\}[,$\n]'

properties = dict(re.findall(properties_rgx, entry_string))
properties

{'title': 'Template morphology and the direction of association',
 'author': 'Yip, Moira',
 'journal': 'Natural {Language} \\& {Linguistic} {Theory}',
 'volume': '6',
 'number': '4',
 'pages': '551--577',
 'year': '1988',
 'publisher': 'Springer',
 'location': 'New York'}

In [4]:
!pip install pymongo

In [60]:
import json
import pymongo  # официальный коннектор
from bson import ObjectId  # можно и из pymongo взять
from pprint import pprint

client = pymongo.MongoClient('localhost', 27017)

In [61]:
# проверить соединение
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [62]:
client.list_database_names()

['admin', 'atambibki', 'config', 'local']

In [63]:
from utils import *

In [9]:
walk_rootdir('/Users/shikunova/Desktop/uni/moksha', 'log.txt')

In [64]:
db = client["atambibki"]
refs = db["refs"]
# db.createCollection()

In [30]:
import json

with open('example.json', 'r') as f:
    content = json.load(f)

In [40]:
refs.insert_many(content, upsert=True)

TypeError: insert_many() got an unexpected keyword argument 'upsert'

In [65]:
client.list_database_names()

['admin', 'atambibki', 'config', 'local']

In [34]:
refs.count_documents({'name': 'Ulfsbjorninn, Shanti'})

0

In [66]:
refs.count_documents({})

532

In [72]:
list(refs.find({'properties': }))

[{'_id': 'groat1995',
  'type': 'article',
  'properties': {'title': 'English expletives: A minimalist approach',
   'journal': 'Linguistic inquiry',
   'pages': '354--365',
   'year': '1995',
   'publisher': 'JSTOR'}},
 {'_id': 'rizzi1986',
  'type': 'article',
  'properties': {'title': 'Null objects in Italian and the theory of pro',
   'journal': 'Linguistic inquiry',
   'volume': '17',
   'number': '3',
   'pages': '501--557',
   'year': '1986',
   'publisher': 'JSTOR'}},
 {'_id': 'preminger2014',
  'type': 'book',
  'properties': {'title': 'Agreement and its failures',
   'volume': '68',
   'year': '2014',
   'publisher': 'MIT press'}},
 {'_id': 'letuchy2020',
  'type': 'article',
  'properties': {'title': 'Raising and similar phenomena in Russian (mainly based on the behavior of pronouns)',
   'journal': 'Voprosy Jazykoznanija',
   'number': '2',
   'pages': '31--60',
   'year': '2020'}},
 {'_id': 'perlmutter2002',
  'type': 'article',
  'properties': {'title': 'Language-internal

In [39]:
?refs.update_many

In [118]:
list(refs.find({'properties.year': '2004'}))

[{'_id': 'Bailyn2004',
  'type': 'article',
  'properties': {'title': 'Generalized inversion',
   'journal': 'Natural Language \\& Linguistic Theory',
   'year': '2004',
   'volume': '22',
   'number': '1',
   'pages': '1-49',
   'doi': '10.1023/B:NALA.0000005556.40898.a5'}},
 {'_id': 'postal2004',
  'type': 'book',
  'properties': {'title': 'Skeptical linguistic essays',
   'year': '2004',
   'publisher': 'Oxford {University} {Press}',
   'address': 'Oxford/New York'}},
 {'_id': 'bailyn2004',
  'type': 'article',
  'properties': {'title': 'Generalized inversion',
   'journal': 'Natural {Language} \\& {Linguistic} {Theory}',
   'volume': '22',
   'number': '1',
   'pages': '1--50',
   'year': '2004',
   'publisher': 'Springer'}},
 {'_id': 'Postal:2004',
  'type': 'book',
  'properties': {'title': 'Skeptical linguistic essays',
   'year': '2004',
   'publisher': 'Oxford {University} {Press}',
   'address': 'Oxford/New York'}},
 {'_id': 'Cable:2004',
  'type': 'unpublished',
  'propertie

In [135]:
from bson.son import SON
pipeline = [
    {"$unwind": "$properties.year"},
    {"$group": {"_id": "$properties.year", "count": {"$sum": 1}}},
    {"$sort": SON([("_id", -1)])},
]

year_data = [dict(zip(['year', 'count'], list(ele.values()))) for ele in refs.aggregate(pipeline) if ele['_id'].isdigit()]
year_data

[{'year': '2023', 'count': 5},
 {'year': '2022', 'count': 17},
 {'year': '2021', 'count': 26},
 {'year': '2020', 'count': 24},
 {'year': '2019', 'count': 11},
 {'year': '2018', 'count': 36},
 {'year': '2017', 'count': 19},
 {'year': '2016', 'count': 7},
 {'year': '2015', 'count': 13},
 {'year': '2014', 'count': 13},
 {'year': '2013', 'count': 13},
 {'year': '2012', 'count': 10},
 {'year': '2011', 'count': 8},
 {'year': '2010', 'count': 16},
 {'year': '2009', 'count': 5},
 {'year': '2008', 'count': 12},
 {'year': '2007', 'count': 15},
 {'year': '2006', 'count': 18},
 {'year': '2005', 'count': 7},
 {'year': '2004', 'count': 16},
 {'year': '2003', 'count': 10},
 {'year': '2002', 'count': 12},
 {'year': '2001', 'count': 13},
 {'year': '2000', 'count': 10},
 {'year': '1999', 'count': 2},
 {'year': '1998', 'count': 10},
 {'year': '1997', 'count': 3},
 {'year': '1996', 'count': 4},
 {'year': '1995', 'count': 8},
 {'year': '1994', 'count': 5},
 {'year': '1993', 'count': 2},
 {'year': '1992', '

In [139]:
pipeline = [
    {"$unwind": "$type"},
    {"$group": {"_id": "$type", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1)])},
]

type_data = [dict(zip(['type', 'count'], list(ele.values()))) for ele in refs.aggregate(pipeline)]
type_data

[{'type': 'article', 'count': 200},
 {'type': 'book', 'count': 81},
 {'type': 'incollection', 'count': 81},
 {'type': 'inproceedings', 'count': 37},
 {'type': 'unpublished', 'count': 33},
 {'type': 'misc', 'count': 31},
 {'type': 'phdthesis', 'count': 26},
 {'type': 'inbook', 'count': 9},
 {'type': 'booklet', 'count': 6},
 {'type': 'proceedings', 'count': 6},
 {'type': 'mastersthesis', 'count': 6},
 {'type': 'electronic', 'count': 4},
 {'type': 'manual', 'count': 4},
 {'type': 'techreport', 'count': 4},
 {'type': 'mvbook', 'count': 1},
 {'type': 'ieeetranbstctl', 'count': 1},
 {'type': 'unpublushed', 'count': 1},
 {'type': 'control', 'count': 1}]

In [1]:
!pip install pybtex

     |████████████████████████████████| 561 kB 1.5 MB/s eta 0:00:01


In [2]:
from pybtex.database.input import bibtex
parser = bibtex.Parser()
bib_data = parser.parse_file('/Users/shikunova/Desktop/uni/research/russian/comparatives/ref.bib')

In [75]:
[{
    '_id': key,
    'type': val.type,
    'properties': dict(val.fields),
} for key, val in bib_data.entries.items()]

[{'_id': 'pancheva2006',
  'type': 'inproceedings',
  'properties': {'title': 'Phrasal and clausal comparatives in {Slavic}',
   'booktitle': 'Formal approaches to {Slavic} linguistics',
   'volume': '14',
   'pages': '236--257',
   'year': '2006',
   'organization': 'Citeseer'}},
 {'_id': 'hoeksema1983',
  'type': 'article',
  'properties': {'title': 'Negative polarity and the comparative',
   'journal': 'Natural {Language} \\& {Linguistic} {Theory}',
   'volume': '1',
   'number': '3',
   'pages': '403--434',
   'year': '1983',
   'publisher': 'Springer',
   'location': 'Berlin'}},
 {'_id': 'vostrikova2019',
  'type': 'mastersthesis',
  'properties': {'title': 'Phrasal and clausal exceptive-additive constructions crosslinguistically',
   'type': 'Doctoral dissertation',
   'year': '2019',
   'institution': 'University of {Massachusetts} {Amherst}'}},
 {'_id': 'matushansky2008',
  'type': 'inproceedings',
  'properties': {'title': 'More of a good thing: {Russian} synthetic and analyti

In [109]:
ele = next(iter(bib_data.entries.items()))

# 'bibtex_first',
#   'bibtex_first_names',
#   'first',
#   'first_names',
#   'get_part',
#   'get_part_as_text',
#   'last',
#   'last_names',
#   'lineage',
#   'lineage_names',
#   'middle',
#   'middle_names',
#   'prelast',
#   'prelast_names',
#   'rich_first_names',
#   'rich_last_names',
#   'rich_lineage_names',
#   'rich_middle_names',
#   'rich_prelast_names',
#   'style1_re',
#   'style2_re',
#   'valid_roles']]
    
dict(ele[1].persons)['author'][0].

['author', 'editor']

In [117]:
print(*[val.to_string('bibtex') for key, val in bib_data.entries.items()])

@inproceedings{pancheva2006,
    author = "Pancheva, Roumyana",
    title = "Phrasal and clausal comparatives in {Slavic}",
    booktitle = "Formal approaches to {Slavic} linguistics",
    volume = "14",
    pages = "236--257",
    year = "2006",
    organization = "Citeseer"
}
 @article{hoeksema1983,
    author = "Hoeksema, Jack",
    title = "Negative polarity and the comparative",
    journal = "Natural {Language} \\& {Linguistic} {Theory}",
    volume = "1",
    number = "3",
    pages = "403--434",
    year = "1983",
    publisher = "Springer",
    location = "Berlin"
}
 @mastersthesis{vostrikova2019,
    author = "Vostrikova, Ekaterina",
    title = "Phrasal and clausal exceptive-additive constructions crosslinguistically",
    type = "Doctoral dissertation",
    year = "2019",
    institution = "University of {Massachusetts} {Amherst}"
}
 @inproceedings{matushansky2008,
    author = "Matushansky, Ora",
    title = "More of a good thing: {Russian} synthetic and analytic comparati

In [58]:
import streamlit as st

st.title('who needs')

st.image('images/citavi.png')
st.image('images/zotero.png')
st.image('images/jabref.png')

st.header('when you can have')

st.image('images/python.png')
st.image('images/mongodb.png')

st.run()

AttributeError: module 'streamlit' has no attribute 'run'